In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px #visualize interactive image
import ace_tools_open as tools;

In [2]:
market_data = pd.read_csv("data/market_data.csv", index_col='Date', parse_dates=True)
vix_data = pd.read_csv("data/vix_data.csv", index_col='Date', parse_dates=True)
macro_data = pd.read_csv("data/macro_monthly.csv", index_col='Date', parse_dates=True)
RIR_data = pd.read_csv("data/real_interest_rate.csv", index_col='Date', parse_dates=True)
usd_data = pd.read_csv("data/usd_index.csv", index_col='Date', parse_dates=True)

gpr_data = pd.read_excel("data/data_gpr_export.xls", header = 0)
gpr_data = gpr_data.rename(columns={"month": "Date"})
gpr_data = gpr_data.set_index("Date")
gpr_selected = gpr_data[["GPR", "GPRT", "GPRA"]]

Resample GPR and macro_data to daily with ffill

In [3]:
gpr_daily = gpr_selected.resample('D').ffill()
macro_daily = macro_data.resample('D').ffill()

combined_data = market_data \
    .join(usd_data, how='inner') \
    .join(macro_daily, how='inner') \
    .join(RIR_data, how='inner') \
    .join(vix_data, how='inner') \
    .join(gpr_daily, how='inner')


tools.display_dataframe_to_user(name="Combined dataFrame", dataframe=combined_data)
combined_data.to_csv("data/combined_data.csv")

Combined dataFrame


Loading ITables v2.4.2 from the internet... (need help?)


# 1. Understand - Explore data

In [4]:
df = combined_data.copy()

In [5]:
df.info

<bound method DataFrame.info of             Crude_Oil  Gold_Futures    Gold_ETF  Silver_Fututes      S&P_500  \
Date                                                                           
2006-01-03  63.139999    530.700012   53.119999        9.087000  1268.800049   
2006-01-04  63.419998    533.900024   53.299999        9.102000  1273.459961   
2006-01-05  62.790001    526.299988   52.340000        8.809000  1273.479980   
2006-01-06  64.209999    539.700012   53.720001        9.112000  1285.449951   
2006-01-09  63.500000    549.099976   54.599998        9.222000  1290.150024   
...               ...           ...         ...             ...          ...   
2024-11-22  71.239998   2709.899902  249.839996       31.309000  5969.339844   
2024-11-25  68.940002   2616.800049  242.479996       30.209999  5987.370117   
2024-11-26  68.769997   2620.300049  242.949997       30.388000  6021.629883   
2024-11-27  68.720001   2639.899902  243.490005       30.111000  5998.740234   
2024-11-

Comment:
- All columns are numericals cols

In [6]:
df.columns

Index(['Crude_Oil', 'Gold_Futures', 'Gold_ETF', 'Silver_Fututes', 'S&P_500',
       'NASDAQ', 'USD_Index', 'CPI', 'Fed_Funds_Rate', 'M2_Supply',
       'Unemployment', 'Treasury_Yield_10Y', 'Real_Interest_Rate', '^VIX',
       'GPR', 'GPRT', 'GPRA'],
      dtype='object')

In [29]:
# df.nunique()

In [7]:
df.isnull().sum()

Crude_Oil              4
Gold_Futures           5
Gold_ETF               0
Silver_Fututes         4
S&P_500                0
NASDAQ                 0
USD_Index             61
CPI                    0
Fed_Funds_Rate         0
M2_Supply              0
Unemployment           0
Treasury_Yield_10Y     0
Real_Interest_Rate    35
^VIX                   0
GPR                    0
GPRT                   0
GPRA                   0
dtype: int64

## 2.Pre-processing